In [1]:
!pip3 install mesa

     |████████████████████████████████| 668 kB 4.9 MB/s 
     |████████████████████████████████| 60 kB 4.2 MB/s 


In [2]:
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import random

import time
import datetime

In [12]:
def obtener_habitacion(modelo):
    habitacion = np.zeros((modelo.grid.width, modelo.grid.height))
    for celda in modelo.grid.coord_iter():
        contenido_celda, x, y = celda
        for contenido in contenido_celda:
            if isinstance(contenido, SortingRobot):
                habitacion[x][y] = 2
            else:
                habitacion[x][y] = contenido.status
    return habitacion
    
class Box(Agent):

    TAKEN = 0
    STACKED = 1
    
    def __init__(self, pos, modelo, status=TAKEN):
        super().__init__(pos, modelo)
        self.x, self.y = pos
        self.stack = 1
        self.status = status
        self.newpos = None
        self.next_status = 0

class SortingRobot(Agent):
    
    def __init__(self, id_unico, modelo):
        super().__init__(id_unico, modelo)
        self.newpos= None
        self.total_moves = 0
        self.container = []
        self.isCarrying = False 

    def step(self):

      neighbors = self.model.grid.get_neighbors(
          self.pos,
          moore=False,
          include_center=False
      )

      for neighbor in neighbors: 

        if isinstance(neighbor, Box):
          if (neighbor.stack < 5) and (neighbor.stack >= 1) and (self.isCarrying): 
            self.container.remove(self.container[0])
            neighbor.stack += 1 
            self.isCarrying = False

          elif (neighbor.stack < 2) and (not self.isCarrying): 
            self.container.append(neighbor)
            neighbor.next_status = neighbor.TAKEN
            self.isCarrying = True 
    
    def advance(self): 

      if self.isCarrying: 
        self.container[0].status = self.container[0].next_status
        
      #Move the robot 
      possible_steps = self.model.grid.get_neighborhood(
          self.pos, 
          moore=True,
          include_center=False
      )

      self.newpos = self.random.choice(possible_steps)
      self.model.grid.move_agent(self, self.newpos)



class Habitacion(Model):
    
    def __init__(self, m, n, num_agentes, num_cajas):
        self.num_agentes = num_agentes
        self.num_cajas = num_cajas 
        self.grid = MultiGrid(m, n, True)
        self.schedule = SimultaneousActivation(self)
               
        
        lista_celdas_vacias = list(self.grid.empties)
        for celdas in range(num_cajas):
            celda_vacia = random.choice(lista_celdas_vacias)
            piso = Box(celda_vacia, self)
            piso.status = piso.STACKED
            self.grid.place_agent(piso, celda_vacia)
            self.schedule.add(piso)
            lista_celdas_vacias.remove(celda_vacia)

        lista_celdas_vacias = list(self.grid.empties) 
        for i in range(num_agentes):
            empty_cell = random.choice(lista_celdas_vacias)
            newrobot = SortingRobot(empty_cell, self)
            self.grid.place_agent(newrobot, empty_cell)
            self.schedule.add(newrobot)

        self.colectordatos = DataCollector(
            model_reporters={'Habitacion': obtener_habitacion},
            agent_reporters={'Movimientos': lambda a: getattr(a, 'total_moves', None)}
        )
    
    def step(self):
        self.colectordatos.collect(self)
        self.schedule.step()    

In [13]:
# Datos de la habitacion:
M = 15
N = 15

NUM_AGENTS = 5

# Porcentaje de celdas inicialmente sucias:
NUM_CAJAS = 40

# Tiempo máximo de ejecución (segundos)
TIEMPO_MAXIMO_EJECUCION = 0.06

start_time = time.time()
tiempo_inicio = str(datetime.timedelta(seconds=TIEMPO_MAXIMO_EJECUCION))
modelo = Habitacion(M, N, NUM_AGENTS, NUM_CAJAS)

while((time.time() - start_time) < TIEMPO_MAXIMO_EJECUCION):
    modelo.step()
    print(str(datetime.timedelta(seconds=(time.time() - start_time))))

# Imprimimos el tiempo que le tomó correr al modelo.
tiempo_ejecucion = str(datetime.timedelta(seconds=(time.time() - start_time)))

In [14]:
todas_habitaciones = modelo.colectordatos.get_model_vars_dataframe()

In [15]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(todas_habitaciones.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(todas_habitaciones.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(todas_habitaciones))

In [16]:
anim